Business Problem

Microsoft wants to join in on the fun of creating original video content. They created a movie studio but don't 
know where to start therefore they have tasked us with coming up some meaningful insights from some movie data as they want to break into the video content business.

The goal is to translate those insights into data that can help the CEO of Microsoft's new studio to make good business
decisions.

To help them we need to use exploratory data analysis to answer the questions below:

1. What genre of movies are doing good in the market right now?
2. How much influence does the budget of a movie have over its success?
3. How does the runtime of a movie influence its success?

Data Understanding

For this analysis we will be using data from IMDB that contains information about a movie's genre,title,
production budget,gross revenue and other things.

Data Overview

We first need to look at the data before doing any changes to it.

In [2]:
import pandas as pd
import matplotlib as plt
import sqlite3

In [5]:
con = sqlite3.connect('Data/im.db')
pd.read_sql("""
SELECT *
FROM sqlite_master
;""",con)

,type,name,tbl_name,rootpage,sql
0,table,movie_basics,movie_basics,2,"CREATE TABLE ""movie_basics"" (\n""movie_id"" TEXT..."
1,table,directors,directors,3,"CREATE TABLE ""directors"" (\n""movie_id"" TEXT,\n..."
2,table,known_for,known_for,4,"CREATE TABLE ""known_for"" (\n""person_id"" TEXT,\..."
3,table,movie_akas,movie_akas,5,"CREATE TABLE ""movie_akas"" (\n""movie_id"" TEXT,\..."
4,table,movie_ratings,movie_ratings,6,"CREATE TABLE ""movie_ratings"" (\n""movie_id"" TEX..."
5,table,persons,persons,7,"CREATE TABLE ""persons"" (\n""person_id"" TEXT,\n ..."
6,table,principals,principals,8,"CREATE TABLE ""principals"" (\n""movie_id"" TEXT,\..."
7,table,writers,writers,9,"CREATE TABLE ""writers"" (\n""movie_id"" TEXT,\n ..."


In [12]:
#Load all the data from all the sources

movie_basics = pd.read_sql("""
SELECT *
FROM movie_basics
;""",con)

movie_ratings = pd.read_sql("""
SELECT *
FROM movie_ratings
;""",con)

budgets = pd.read_csv('Data/tn.movie_budgets.csv')
budgets

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
2,3,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350"
3,4,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963"
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747"
...,...,...,...,...,...,...
5777,78,"Dec 31, 2018",Red 11,"$7,000",$0,$0
5778,79,"Apr 2, 1999",Following,"$6,000","$48,482","$240,495"
5779,80,"Jul 13, 2005",Return to the Land of Wonders,"$5,000","$1,338","$1,338"
5780,81,"Sep 29, 2015",A Plague So Pleasant,"$1,400",$0,$0


In [17]:
# Getting the data together
movie_imdb = movie_basics.merge(movie_ratings, how = 'right', on='movie_id')
movie_imdb

,movie_id,primary_title,original_title,start_year,runtime_minutes,genres,averagerating,numvotes
0,tt10356526,Laiye Je Yaarian,Laiye Je Yaarian,2019,117.0,Romance,8.3,31
1,tt10384606,Borderless,Borderless,2019,87.0,Documentary,8.9,559
2,tt1042974,Just Inès,Just Inès,2010,90.0,Drama,6.4,20
3,tt1043726,The Legend of Hercules,The Legend of Hercules,2014,99.0,"Action,Adventure,Fantasy",4.2,50352
4,tt1060240,Até Onde?,Até Onde?,2011,73.0,"Mystery,Thriller",6.5,21
...,...,...,...,...,...,...,...,...
73851,tt9805820,Caisa,Caisa,2018,84.0,Documentary,8.1,25
73852,tt9844256,Code Geass: Lelouch of the Rebellion - Glorifi...,Code Geass: Lelouch of the Rebellion Episode III,2018,120.0,"Action,Animation,Sci-Fi",7.5,24
73853,tt9851050,Sisters,Sisters,2019,NaN,"Action,Drama",4.7,14
73854,tt9886934,The Projectionist,The Projectionist,2019,81.0,Documentary,7.0,5
